In [10]:
from sklearn.ensemble import RandomForestClassifier
from sklearn import preprocessing
from sklearn import metrics
import pandas as pd
import numpy as np

test = pd.read_csv("test.csv")
train = pd.read_csv("train.csv")

train_label = pd.DataFrame(train.Activity)
train = train.drop(['Activity','subject'], axis=1)

test_label = pd.DataFrame(test.Activity)
test = test.drop(['Activity','subject'], axis=1)

encoder = preprocessing.LabelEncoder()
encoder.fit(train_label)
train_label = encoder.transform(train_label)
encoder.fit(test_label)
test_label = encoder.transform(test_label)

def build_visited(n, m):
    """
    >>> build_visited(3, 3)
    [[False, False, False], [False, False, False], [False, False, False]]
    """
    grid = []
    for i in range(0, n):
        grid.append([False for item in range(0, m)])
    return grid


def n_estimator(n, m):

    clf=RandomForestClassifier(n_estimators=n, max_depth=m)
    clf.fit(train,train_label)
    y_pred = clf.predict(test)
    return metrics.accuracy_score(test_label, y_pred)

def get_children(x, y, n, m):

    children = []
    for dx, dy in [(0, -1), (0, 1), (-1, 0), (1, 0)]:
        cx, cy = (x + dx), (y + dy)
        if 0 <= cx and cx < n and 0 <= cy and cy < m:
            children.append((cx, cy))
    return children


def search_best_accuracy(n_values, m_values, baseline):

    n_size, m_size = [len(values) for values in (n_values, m_values)]
    if n_size == 0 or m_size == 0 or baseline > 1: return ()
    visited = build_visited(n_size, m_size)
    best = [(), 0.0, True]
    queue = [(0, 0)]
    while 0 < len(queue):
        current = queue.pop(0)
        x, y = current
        n, m = n_values[x], m_values[y]
        estimated = n_estimator(n, m)
        if estimated >= baseline: return [(n, m), estimated, True]
        if estimated >= best[1]: best = [(n, m), estimated, False]
        visited[x][y] = True
        children = get_children(x, y, n_size, m_size)
        for cx, cy in children:
            if not visited[cx][cy]: queue.append((cx, cy))
    visited[x][y] = True
    print(visited)
    return best


def draw_graph(n,m):  
    from graphviz import Graph, Digraph
    bfs_graph = Digraph('bfs', filename='bfs.gv', format='png',node_attr={'color': 'lightblue2'}, engine='dot')
    bfs_graph.attr('node', shape='circle')

    all_pos = []
    for i in n:
        for j in m:
            all_pos.append('n={} / m={}'.format(i,j))

    nodes_from = []
    nodes_to = []
    for i in all_pos:
        for j in all_pos:
            if j != all_pos[0] and i != j:
                nodes_from.append(i)
                nodes_to.append(j)
            elif i != all_pos[0] and j != i:
                nodes_from.append(j)
                nodes_to.append(i)

    edges = list(set(list(zip(nodes_from,nodes_to))))
    bfs_graph.edges(edges)
    
    return bfs_graph.render('bfs_graph.gv', view=True)

if __name__ == "__main__":
    import doctest
    doctest.testmod()


n = [30,40,50,60]
m = [10,20,30,23]

print("The best parameters with the accuracy result: ",search_best_accuracy(n,m,0.9))

print("\n\n The graph for the estimators: ",draw_graph(n,m))

/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


The best parameters with the accuracy result:  [(30, 10), 0.9229725144214456, True]


ExecutableNotFound: failed to execute ['dot', '-Tpng', '-O', 'bfs_graph8.gv'], make sure the Graphviz executables are on your systems' PATH